In [1]:
'''
Author: Sriram Yarlagadda + George Nakhleh
Date Created: 10/15/2016
Note: Please use Python 2.7
'''
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir('C:\Users\syarlag1\Desktop\Loyola-SAH-Seasonality-Analysis')
%matplotlib inline

### Reading Data

In [2]:
os.chdir('./data')
dataFiles = os.listdir('.')
allDataDict = {}
# list of variables we need
varList = ['SCORE','SPRING','SUMMER','WINTER','FALL','FEMALE']
for dataFile in dataFiles:
    state = dataFile[-6:-4]
    allDataDict[state] = pd.read_stata(dataFile, columns=varList)

In [3]:
for key in allDataDict.keys(): #creating a new state identifier column
    allDataDict[key]['State'] = pd.Series([str(key)[:2]]*allDataDict[key].shape[0])
allStateData = pd.concat(allDataDict.values()) #combing into a single dataframe

In [4]:
allStateData.index = range(allStateData.shape[0])

### Data Preprocessing

In [5]:
allStateData.shape

(29708, 7)

In [6]:
allStateData.columns

Index([u'SCORE', u'SPRING', u'SUMMER', u'WINTER', u'FALL', u'FEMALE',
       u'State'],
      dtype='object')

In [7]:
np.sum(np.array(allStateData < 0), 0) # 88 values < 0 (invalid)

array([ 0,  0,  0,  0,  0, 88,  0])

In [8]:
np.sum(np.array(pd.isnull(allStateData)),0) # 226 NaNs

array([  0,   0,   0,   0,   0, 226,   0])

In [9]:
for feature in allStateData.columns: 
    allStateData = allStateData[allStateData[feature] >= 0]

In [10]:
# removing instances where all months are equal to 0
allStateData = allStateData.drop(allStateData[allStateData['FALL']+\
                                                  allStateData['WINTER']+\
                                                  allStateData['SUMMER']+\
                                                  allStateData['SPRING']== 0].index)

In [11]:
allStateData.shape

(29304, 7)

In [13]:
# creating a new "season" variable
allStateData.index = range(allStateData.shape[0]) # reindexing again to account for the removed rows
season = []
for iRow in range(allStateData.shape[0]):
    if allStateData['FALL'][iRow] == 1: season.append('FALL')
    if allStateData['SPRING'][iRow] == 1: season.append('SPRING')
    if allStateData['SUMMER'][iRow] == 1: season.append('SUMMER')
    if allStateData['WINTER'][iRow] == 1: season.append('WINTER')

allStateData['SEASON'] = pd.Series(season)

In [14]:
# saving an excel copy of the entire data for later use
os.chdir('./..')
allStateData.to_csv('allData.csv') # note that all data only includes the column subsets we need

### Seasonality

#### *Question*: Is there seasonality in the SAH Scores?

Let us first perform some quick data exploration

In [22]:
#count for each season
allStateData.SEASON.value_counts()

SPRING    7497
FALL      7450
WINTER    7390
SUMMER    6967
Name: SEASON, dtype: int64

In [18]:
# barplots of counts for each season
pd.pivot(allStateData['SEASON'],allStateData['SCORE'],)

ValueError: Index contains duplicate entries, cannot reshape